In [1]:
import pandas as pd
import duckdb

In [2]:
with duckdb.connect('../data/avito_builder_db.duckdb') as w:
    builder_df = w.execute('select * from INTEL.avito_RE ').df()
with duckdb.connect('../data/avito_user_db.duckdb') as w:
    user_df = w.execute('select * from INTEL.avito_RE ').df()
with duckdb.connect('../data/avito_agency_db.duckdb') as w:
    agency_df = w.execute('select * from INTEL.avito_RE ').df()

In [11]:
builder_df.groupby(pd.qcut(builder_df['m2'],3))['price'].describe().div(1000).round(3).style.format('{:_}')

,count,mean,std,min,25%,50%,75%,max
m2,,,,,,,,
"(35.999, 53.0]",0.12,12_777.653,2_384.651,9_641.05,11_070.318,12_220.642,13_891.79,26_287.814
"(53.0, 69.0]",0.12,18_080.909,5_946.942,11_087.146,14_662.603,15_998.069,18_426.157,37_196.926
"(69.0, 138.0]",0.116,25_513.722,12_563.312,12_040.6,16_872.686,18_762.358,30_612.2,57_989.833


In [12]:
user_df.groupby(pd.qcut(user_df['m2'],3))['price'].describe().div(1000).round(3).style.format('{:_}')

,count,mean,std,min,25%,50%,75%,max
m2,,,,,,,,
"(30.999, 53.0]",0.939,14_061.265,4_056.797,5_321.112,11_500.0,13_100.0,15_500.0,34_900.0
"(53.0, 65.0]",0.938,17_221.246,5_372.587,5_300.0,13_900.0,16_000.0,19_287.5,57_000.0
"(65.0, 190.0]",0.925,26_617.334,14_366.178,7_896.0,18_100.0,23_000.0,30_750.0,200_000.0


In [13]:
agency_df.groupby(pd.qcut(agency_df['m2'],3))['price'].describe().div(1000).round(3).style.format('{:_}')

,count,mean,std,min,25%,50%,75%,max
m2,,,,,,,,
"(31.999, 58.0]",0.469,16_186.328,7_112.189,5_270.673,11_800.0,14_250.0,18_450.0,61_000.0
"(58.0, 79.0]",0.439,26_476.056,20_760.077,8_900.0,16_000.0,21_000.0,29_945.0,192_460.0
"(79.0, 254.0]",0.445,71_767.205,67_578.796,14_500.0,32_999.0,46_500.0,78_009.0,377_328.328


In [113]:
connect =  duckdb.connect('../data/general_moscow_home_info.duckdb')
base_home_info = connect.execute('select * from home_base_info;').df()
connect.close()

In [151]:
base_home_info.head()

,Город,Адрес,Площадь м2,Год,Этажей,home_url,page,id,Год ввода в эксплуатацию,Детская площадка,...,Запорная арматура системы отопления,Отопительные приборы,Тип отопительных приборов,Сеть внутридомовой инженерной системы холодного водоснабжения,Стояки системы холодного водоснабжения,Запорная арматура системы холодного водоснабжения,Сеть внутридомовой инженерной системы горячего водоснабжения,Стояки системы горячего водоснабжения,Запорная арматура системы горячего водоснабжения,Дата установления класса энергоэффективности
0,Москва,"1-й Амбулаторный проезд, 2",10046.4,1968,9,https://dom.mingkh.ru/moskva/moskva/916691,1,916691.0,1968.0,Есть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Москва,"1-й Амбулаторный проезд, 5 корпус 1",2525.8,1959,5,https://dom.mingkh.ru/moskva/moskva/422302,1,422302.0,1959.0,Есть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Москва,"1-й Амбулаторный проезд, 5 корпус 2",5129,1962,5,https://dom.mingkh.ru/moskva/moskva/422303,1,422303.0,1962.0,Есть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Москва,"1-й Амбулаторный проезд, 7 корпус 3",7179.6,1974,12,https://dom.mingkh.ru/moskva/moskva/422304,1,422304.0,1974.0,Есть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Москва,"1-й Амбулаторный проезд, 7 корпус А",2461.2,1957,5,https://dom.mingkh.ru/moskva/moskva/422305,1,422305.0,1957.0,Есть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
with duckdb.connect('../data/avito_builder_db.duckdb') as w:
    builder_df_base = w.execute('select * from RAW.avito_RE ').df()
with duckdb.connect('../data/avito_user_db.duckdb') as w:
    user_df_base = w.execute('select * from RAW.avito_RE ').df()
with duckdb.connect('../data/avito_agency_db.duckdb') as w:
    agency_df_base = w.execute('select * from RAW.avito_RE ').df()

In [164]:

def featuring_data(item_list:list)->pd.DataFrame:
    data = pd.DataFrame(item_list)
    data['price'] = data['price'].astype(float)
    data['street']= data['adress'].str.extract('(.*?), (?=\d.*)')
    data['is_new']= data['JK'] == ''
    data['n_rooms']=data['title'].str.extract('«(.*?),')
    data['m2'] = data['title'].str.extract(', (\d+).*м²').astype(float)
    data[['floor','max_floor']] = data['title'].str.extract('(\d+/\d+).*эт')[0].str.split('/',expand=True).astype(float)
    data['text'] = data['text'].str.replace('\n','')
    data['rubm2'] = data['price'] / data['m2']
    data.drop(['title'],axis=1,inplace=True)
    return data

In [167]:
new_builder = featuring_data(builder_df_base)
new_user = featuring_data(user_df_base)
new_agency = featuring_data(agency_df_base)

In [168]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    w.execute('''
                CREATE TABLE user_ads as select * from new_user;
                CREATE TABLE builder_ads as select * from new_builder;
                CREATE TABLE agency_ads as select * from new_agency;
                CREATE TABLE home_info as select * from base_home_info;
            ''')
    print(w.execute('PRAGMA show_tables').df())

          name
0   agency_ads
1  builder_ads
2    home_info
3     user_ads


In [170]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    print(w.execute('PRAGMA show_tables').df()['name'])

0     agency_ads
1    builder_ads
2      home_info
3       user_ads
Name: name, dtype: object


In [176]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    for table in w.execute('PRAGMA show_tables').df()['name'].values:

        print(table,' :',w.execute(f'select count(*) as events from {table}').df().iloc[0].values)

agency_ads  : [3165]
builder_ads  : [3177]
home_info  : [32933]
user_ads  : [4117]
